# Importing packages 

Selenium package is used for automating web browser interaction with python.

In [1]:
#Import Packages
from selenium import webdriver
import time
import pandas as pd
import os
#Import Packages

from datetime import datetime
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [2]:
# !pip install selenium
# !pip install pandas

In [3]:
url1='https://www.linkedin.com/jobs/search/?keywords=Data%20Analyst&location=United%20States&locationId=&geoId=103644278&f_TPR=r604800&f_E=2%2C3&f_PP=104194190&position=1&pageNum=0'

# service = Service('C:\\Users\\suraj\\OneDrive - UNT System\\2023\\softwares\\chromedriver_win32\\chromedriver.exe')
options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(service=service, options=options)
driver = webdriver.Chrome(options = options)
driver.maximize_window()
driver.implicitly_wait(3)
driver.get(url1)

Finding the number of the jobs by extracting from linkedin json file and after enough data cleaning.

In [4]:
y=driver.find_elements(By.CLASS_NAME,'results-context-header__job-count')[0].text
y=y.strip('+')
y=y.replace(",", "")
y=pd.to_numeric(y)
y

233

Code to scroll the linkedin screen to fetch the job details

In [5]:
i=1                                     # adjusting scroll_position = 1 
time.sleep(4)                           # waiting for 4 sec for page to load

scroll_pause_time = 1                   
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the webpage

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1   # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    time.sleep(scroll_pause_time)      # Buffer time for pausing after scrolling
   
    scroll_height = driver.execute_script("return document.body.scrollHeight;") # fetching the new height of the screen
    try:
        send=driver.find_element(By.XPATH,"//button[@aria-label='See more jobs']") # clicking the see more jobs button everytime it appears
        driver.execute_script("arguments[0].click();", send)   
        time.sleep(3)                                          # Buffer time after clicking button to load the page
    except:
        pass
        time.sleep(1)                                     # if button doesn't appear just wait for 1 sec after scrolling
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break                               


In [6]:
#Create empty lists for company name and job title
companyname= []
#Find company name and append it to the blank list

for j in range(y):
    try:
        company=driver.find_elements(By.CLASS_NAME,'base-search-card__subtitle')[j].text
        companyname.append(company)
 
    except :
        pass
 

In [7]:
#Create empty lists for company name and job title
titlename= []

#Find position name and append it to the blank list
for i in range(y):
    try:
        title=driver.find_elements(By.CLASS_NAME,'base-search-card__title')[i].text
        titlename.append(title) 
    except :
        pass
 

In [8]:
# Number of records in companyname list
len(companyname)

233

In [10]:
# Number of records in titlename list
len(titlename)

233

In [11]:
# Using pandas to transform the lists like company and title into dataframes
companyname=pd.DataFrame(companyname,columns=["Company"])
titlename=pd.DataFrame(titlename,columns=['Title'])

In [12]:
# concatinating both companyname and titlename dataframes horizontally
combined_df=pd.concat([companyname,titlename],axis=1)
combined_df

,Company,Title
0,"TekVivid, Inc",Data Analyst
1,"TekVivid, Inc",Data Analyst
2,Nenni and Associates,Data Analyst
3,LP Analyst,Document Administrator
4,Robert Half,Data Analyst
...,...,...
228,Dice,Java developer
229,Dice,"Senior UI Developer / Dallas, TX"
230,Aha!,Ruby on Rails Engineer
231,Dice,"Senior UI and React Developer / Dallas, TX"


In [13]:
# fetching the job links and appending it into a list 
jobList=driver.find_elements(By.CLASS_NAME,"base-card__full-link")
hrefList = []
for e in jobList:
    hrefList.append(e.get_attribute('href'))

In [15]:
#Created empty lists for postings and last updated and appending it

last_updated= []
postdate=[]

for d in range(y):
    try:
        posting=driver.find_elements(By.CLASS_NAME,'job-search-card__listdate')[d].text
        datetime_value = driver.find_elements(By.CLASS_NAME,'job-search-card__listdate')[d].get_attribute('datetime')
        postdate.append(datetime_value)
        last_updated.append(posting)
    except :
        pass 

In [16]:
postdate=pd.DataFrame(postdate,columns=["postdate"])
last_updated=pd.DataFrame(last_updated,columns=["last_updated"])

In [17]:
dt=datetime.now().strftime("%Y-%m-%d %H-%M-%S")
print(dt)

2023-07-16 10-17-14


In [18]:
# concatinating all dataframes horizontally
linklist=pd.DataFrame(hrefList,columns=["joblinks"])
combined_df = pd.concat([postdate,combined_df,linklist,last_updated], axis=1)
combined_df
filename= 'linkedIn_jobs'+ dt+'.csv'
combined_df.to_csv(filename,index=False)

#Close the driver
driver.close()